# LSTMを用いたテキスト分類
【概要】

データセットはlivedoor ニュースコーパスを使用．
各ニュース記事は９種類のカテゴリに分類されている．
ニュース記事のタイトルを入力とし，推測したカテゴリを出力する．

GPUを利用するため，Google colaboratory上で実行．

実行する前に，編集→ノートブックの設定でGPUを選択する．

データの流れは以下の通り．

ニュース記事からタイトル文字列を抽出

↓

形態素解析を行い，文字列を形態素に分割し，各形態素をIDに変換

↓

エンべディング層に入力

↓

LSTM層に入力

↓

Affine層に入力

↓

Softmax層に入力

↓

カテゴリの確率を出力



---


【工夫点】

１．過学習が疑われるため，ドロップアウトを適用

２．タイトル+本文を入力データとして利用

３．LSTM層の多層化，Bidirectional化によるモデルの改良

---



In [ ]:
# Googleドライブ上のファイルにアクセスするためのマウント設定
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install mecab-python3==0.996.5

データセットのフォルダにアクセスし，データフレームを作成する．
各列のラベルはそれぞれ，タイトル，タイトル+本文，カテゴリー．

In [ ]:
#import os
#from glob import glob
#import pandas as pd
#import linecache

#drive_dir = "drive/My Drive/自作課題/"

#categories = [name for name in os.listdir(drive_dir + 'text') if os.path.isdir(drive_dir + "text/" +name)]
#print(categories)

#datasets = pd.DataFrame(columns=["title", "text", "category"])
#for cat in categories:
#    path = drive_dir + "text/" + cat + "/*.txt"
#    files = glob(path)
#    for file in files:
#        with open(file, 'r') as f:
#            lines = f.read().splitlines()
#            title = lines[2]
#            body = "\n".join(lines[3:])
#            text = title + "\n" + body
#        s = pd.Series([title, text, cat], index=datasets.columns)
#        datasets = datasets.append(s, ignore_index=True)
#datasets.head()

↑を毎回実行すると時間がかかるため，一度作成したデータフレームを保存して再利用する．

In [ ]:
import os
import pandas as pd

drive_dir = "drive/My Drive/自作課題/"

datasets = pd.read_csv(drive_dir + "text.csv")
categories = [name for name in os.listdir(drive_dir + 'text') if os.path.isdir(drive_dir + "text/" +name)]
print(categories)
datasets.head()

['dokujo-tsushin', 'smax', 'livedoor-homme', 'kaden-channel', 'it-life-hack', 'sports-watch', 'peachy', 'topic-news', 'movie-enter']


,Unnamed: 0,title,text,category
0,0,コンプレックスを自信に変える女たち,コンプレックスを自信に変える女たち\nどんな人でもコンプレックスを持っているのではないか。\...,dokujo-tsushin
1,1,親が倒れた時、独女は何ができるのか？,親が倒れた時、独女は何ができるのか？\n「親ももう60歳すぎているし、いつ何があってもおかし...,dokujo-tsushin
2,2,ただの女友達と好きな女の境界線,ただの女友達と好きな女の境界線\n職場で気が合い、話も弾み、ときにはふたりで食事に行くことも...,dokujo-tsushin
3,3,【オトナ女子映画部】少女マンガ的な妄想ふくらむ“鉄板”ラブコメ『Black & White/...,【オトナ女子映画部】少女マンガ的な妄想ふくらむ“鉄板”ラブコメ『Black & White/...,dokujo-tsushin
4,4,男性の言い分「独女の食事スタイル、ここが納得いかない！」,男性の言い分「独女の食事スタイル、ここが納得いかない！」\n男性と女性は同じ人間でも特性は全...,dokujo-tsushin


文字列を形態素に分解する関数を定義

In [ ]:
import MeCab
import re

tagger = MeCab.Tagger("-Owakati")

def make_wakati(sentence):
    sentence = tagger.parse(sentence)
    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
    wakati = sentence.split(" ")
    wakati = list(filter(("").__ne__, wakati))
    return wakati

タイトル文字列を形態素に分解し，各形態素にIDを振り分け，形態素とIDの対応リストを作成．

In [ ]:
word2index = {}
# 系列を揃えるためのパディング文字列<pad>を追加
# パディング文字列のIDは0とする
word2index.update({"<pad>":0})

for title in datasets["title"]:
    wakati = make_wakati(title)
    for word in wakati:
        if word in word2index:  continue
        word2index[word] = len(word2index)

print("vocab size : ", len(word2index))

vocab size :  13230


学習データのミニバッチを作成するために，各文字列データの系列長を揃える必要がある．

短い系列長のデータにパディングを追加し，最長のデータの系列長に揃える．

In [ ]:
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle

# カテゴリーとIDの対応リストを作成
cat2index = {}
for cat in categories:
    if cat in cat2index: continue
    cat2index[cat] = len(cat2index)

# 入力した文字列を形態素に分解し，各形態素をIDに変換する関数
def sentence2index(sentence):
    wakati = make_wakati(sentence)
    return [word2index[w] for w in wakati]

# 入力したカテゴリーをIDに変換する関数
def category2index(cat):
    return [cat2index[cat]]

index_datasets_title_tmp = []
index_datasets_category = []

# バッチ化のために各文字列データの系列長を揃える
# 系列の長さの最大値を取得。この長さに他の系列の長さをあわせる
max_len = 0
for title, category in zip(datasets["title"], datasets["category"]):
    index_title = sentence2index(title)
    index_category = category2index(category)
    index_datasets_title_tmp.append(index_title)
    index_datasets_category.append(index_category)
    if max_len < len(index_title):
        max_len = len(index_title)

# 系列の長さを揃えるために短い系列にパディングを追加
index_datasets_title = []
for title in index_datasets_title_tmp:
    for i in range(max_len - len(title)):
        title.insert(0, 0) # 前パディング
#      title.append(0)　# 後ろパディング
    index_datasets_title.append(title)

train_x, test_x, train_y, test_y = train_test_split(index_datasets_title, index_datasets_category, train_size=0.8, random_state=11)

# データをバッチでまとめるための関数
def train2batch(title, category, batch_size=32):
    title_batch = []
    category_batch = []
    title_shuffle, category_shuffle = shuffle(title, category)
    for i in range(0, len(title), batch_size):
        title_batch.append(title_shuffle[i:i+batch_size])
        category_batch.append(category_shuffle[i:i+batch_size])
    return title_batch, category_batch

ベースモデル定義

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# GPUを使うために必要
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# ベースモデル
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        # <pad>の単語IDが0なので、padding_idx=0としている
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        # batch_first=Trueとすることで入力テンソルの型を(batch_size × len(sentence) × embedding_dim)にできる
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        self.softmax = nn.LogSoftmax()
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        #embeds.size() = (batch_size × len(sentence) × embedding_dim)
        _, lstm_out = self.lstm(embeds)
        # lstm_out[0].size() = (1 × batch_size × hidden_dim)
        tag_space = self.hidden2tag(lstm_out[0])
        # tag_space.size() = (1 × batch_size × tagset_size)

         # (batch_size × tagset_size)にするために次元削減squeeze()する
        tag_scores = self.softmax(tag_space.squeeze())
         # tag_scores.size() = (batch_size × tagset_size)

        return tag_scores

EMBEDDING_DIM = 200
HIDDEN_DIM = 128
VOCAB_SIZE = len(word2index)
TAG_SIZE = len(categories)
# to(device)でモデルがGPU対応する
model_base = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE).to(device)
loss_function_base = nn.NLLLoss()
optimizer_base = optim.Adam(model_base.parameters(), lr=0.001)
#勾配クリッピングの閾値
max_norm = 5.0


学習

In [ ]:
def trainer(train_x, train_y, model, loss_function, optimizer):
    losses = []
    model.train()
    for epoch in range(50):
        all_loss = 0
        title_batch, category_batch = train2batch(train_x, train_y)
        for i in range(len(title_batch)):
            batch_loss = 0

            model.zero_grad()
            
            # 順伝搬させるtensorはGPUで処理させるためdevice=にGPUをセット
            title_tensor = torch.tensor(title_batch[i], device=device)
            # category_tensor.size() = (batch_size × 1)なので、squeeze()
            category_tensor = torch.tensor(category_batch[i], device=device).squeeze()

            out = model(title_tensor)

            batch_loss = loss_function(out, category_tensor)
            batch_loss.backward()
            # 勾配クリッピング
            #nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            optimizer.step()

            all_loss += batch_loss.item()
        print("epoch", epoch, "\t", "loss", all_loss)
        if all_loss < 0.1: break
    print("done.")

trainer(train_x, train_y, model_base, loss_function_base, optimizer_base)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 0 	 loss 278.3312571644783
epoch 1 	 loss 156.70109927654266
epoch 2 	 loss 89.34506480395794
epoch 3 	 loss 44.169016391038895
epoch 4 	 loss 21.022379517555237
epoch 5 	 loss 9.119648973457515
epoch 6 	 loss 5.243457025848329
epoch 7 	 loss 2.3246719921007752
epoch 8 	 loss 1.4754446597071365
epoch 9 	 loss 1.203811745508574
epoch 10 	 loss 1.0188539265072905
epoch 11 	 loss 0.9150644512847066
epoch 12 	 loss 0.8696114233171102
epoch 13 	 loss 0.7987178986077197
epoch 14 	 loss 0.7742376437236089
epoch 15 	 loss 0.7221189943957143
epoch 16 	 loss 0.691118494287366
epoch 17 	 loss 0.6712544714828255
epoch 18 	 loss 0.6512145747401519
epoch 19 	 loss 0.6481697559793247
epoch 20 	 loss 0.6375674666051054
epoch 21 	 loss 0.5930877310602227
epoch 22 	 loss 0.5968639991406235
epoch 23 	 loss 0.5845328932846314
epoch 24 	 loss 0.5722149166176678
epoch 25 	 loss 0.5805848214949947
epoch 26 	 loss 0.570289440878696
epoch 27 	 loss 0.5622812460824207
epoch 28 	 loss 0.5512585903634317
ep

テストデータによる推論・精度算出

In [ ]:
def accuracy(test_x, test_y, model):
    test_num = len(test_x)
    a = 0
    model.eval()
    with torch.no_grad():
        title_batch, category_batch = train2batch(test_x, test_y)

        for i in range(len(title_batch)):
            title_tensor = torch.tensor(title_batch[i], device=device)
            category_tensor = torch.tensor(category_batch[i], device=device)

            out = model(title_tensor)
            _, predicts = torch.max(out, 1)
            for j, ans in enumerate(category_tensor):
                if predicts[j].item() == ans.item():
                    a += 1
        return a / test_num
model_base_acc = accuracy(test_x, test_y, model_base)
print("model_base accuracy :", model_base_acc)

model_base accuracy : 0.6795392953929539


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


精度があまり良くないので，学習データに対する精度をチェックする

In [ ]:
model_base_acc_train = accuracy(train_x, train_y, model_base)
print("model_base train accuracy :", model_base_acc_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


model_base train accuracy : 0.9988135593220339


学習データに対する精度が99%以上であるため，過学習を起こしていると考えられる．

# 【ドロップアウトの適用】

そこで，過学習を抑制し汎化性能を向上させるため，ドロップアウトを適用したモデルを作成する．

ドロップアウトモデル定義

In [ ]:
# ドロップアウトモデル
class DropLSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size):
        super(DropLSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        # ドロップアウト層の定義
        self.dropout1 = nn.Dropout(0.5)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.dropout2 = nn.Dropout(0.5)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        self.softmax = nn.LogSoftmax()
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        # ドロップアウト層１
        embeds = self.dropout1(embeds)
        _, lstm_out = self.lstm(embeds)
        # ドロップアウト層２
        drop_out = self.dropout2(lstm_out[0])
        tag_space = self.hidden2tag(drop_out)
        tag_scores = self.softmax(tag_space.squeeze())

        return tag_scores

model_drop = DropLSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE).to(device)
loss_function_drop = nn.NLLLoss()
optimizer_drop = optim.Adam(model_drop.parameters(), lr=0.001)

ドロップアウトモデル学習

In [ ]:
trainer(train_x, train_y, model_drop, loss_function_drop, optimizer_drop)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 0 	 loss 347.0742018222809
epoch 1 	 loss 263.77784419059753
epoch 2 	 loss 224.72939485311508
epoch 3 	 loss 198.8028565645218
epoch 4 	 loss 176.46608859300613
epoch 5 	 loss 154.0020693540573
epoch 6 	 loss 140.04406183958054
epoch 7 	 loss 125.7641934454441
epoch 8 	 loss 112.59957931935787
epoch 9 	 loss 105.65650929510593
epoch 10 	 loss 93.74145624041557
epoch 11 	 loss 89.28340162336826
epoch 12 	 loss 79.0368916541338
epoch 13 	 loss 73.46407886594534
epoch 14 	 loss 68.80392841249704
epoch 15 	 loss 65.3428254276514
epoch 16 	 loss 58.01862517744303
epoch 17 	 loss 55.64290173724294
epoch 18 	 loss 52.02083423361182
epoch 19 	 loss 48.236286997795105
epoch 20 	 loss 45.80949308723211
epoch 21 	 loss 43.89563933387399
epoch 22 	 loss 39.43948703631759
epoch 23 	 loss 38.183571204543114
epoch 24 	 loss 34.30421833693981
epoch 25 	 loss 33.58682820573449
epoch 26 	 loss 30.886242234148085
epoch 27 	 loss 30.246362840756774
epoch 28 	 loss 29.766367103904486
epoch 29 	 loss

ドロップアウトモデル推論・精度算出

In [ ]:
model_drop_acc = accuracy(test_x, test_y, model_drop)
print("model_base accuracy :", model_base_acc)
print("model_drop accuracy :", model_drop_acc)

model_base accuracy : 0.6795392953929539
model_drop accuracy : 0.7730352303523035


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


精度がやや向上した．

ドロップアウトの適用により過学習を抑制する正則化が効いていると考えられる．

# 【タイトル＋本文を入力データにしてみる】

続いて，タイトル+本文を入力データとした場合に精度が向上するか調査する．

タイトル+本文のデータセットを構築

In [ ]:
word2index = {}
word2index.update({"<pad>":0})

for title in datasets["text"]:
    wakati = make_wakati(title)
    for word in wakati:
        if word in word2index:  continue
        word2index[word] = len(word2index)

print("vocab size : ", len(word2index))

vocab size :  59930


In [ ]:
index_datasets_text_tmp = []
index_datasets_category = []

# 系列の長さの最大値を取得。この長さに他の系列の長さをあわせる
max_len = 0
for text, category in zip(datasets["text"], datasets["category"]):
    index_text = sentence2index(text)
    index_category = category2index(category)
    index_datasets_text_tmp.append(index_text)
    index_datasets_category.append(index_category)
    if max_len < len(index_text):
        max_len = len(index_text)

# 系列の長さを揃えるために短い系列にパディングを追加
index_datasets_text = []
for text in index_datasets_text_tmp:
    for i in range(max_len - len(text)):
        text.insert(0, 0) # 前パディング
#      title.append(0)　# 後ろパディング
    index_datasets_text.append(text)

train_x_full, test_x_full, train_y_full, test_y_full = train_test_split(index_datasets_text, index_datasets_category, train_size=0.8, random_state=11)

モデル定義

先ほどのドロップアウトモデルを利用

In [ ]:
EMBEDDING_DIM = 200
HIDDEN_DIM = 128
VOCAB_SIZE = len(word2index)
TAG_SIZE = len(categories)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model_full = DropLSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE).to(device)
loss_function_full = nn.NLLLoss()
optimizer_full = optim.Adam(model_full.parameters(), lr=0.001)

cuda


タイトル+本文データ学習

In [ ]:
trainer(train_x_full, train_y_full, model_full, loss_function_full, optimizer_full)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 0 	 loss 308.6572422981262
epoch 1 	 loss 204.9768026471138
epoch 2 	 loss 153.79789212346077
epoch 3 	 loss 139.21663016080856
epoch 4 	 loss 124.4324600994587
epoch 5 	 loss 99.41649608314037
epoch 6 	 loss 89.65364748239517
epoch 7 	 loss 75.95142197608948
epoch 8 	 loss 63.41925152763724
epoch 9 	 loss 76.62054872885346
epoch 10 	 loss 63.36690326780081
epoch 11 	 loss 50.78928077034652
epoch 12 	 loss 42.91469419002533
epoch 13 	 loss 40.772968439385295
epoch 14 	 loss 39.07394891232252
epoch 15 	 loss 33.04709302447736
epoch 16 	 loss 31.403015030547976
epoch 17 	 loss 32.72948788944632
epoch 18 	 loss 27.228899087756872
epoch 19 	 loss 25.301443964242935
epoch 20 	 loss 22.68157380167395
epoch 21 	 loss 29.51691563008353
epoch 22 	 loss 22.72107018623501
epoch 23 	 loss 31.833334304392338
epoch 24 	 loss 26.924013352021575
epoch 25 	 loss 19.625721542863175
epoch 26 	 loss 18.177563628181815
epoch 27 	 loss 17.033250245265663
epoch 28 	 loss 16.306104346178472
epoch 29 	 l

タイトル+本文モデル推論・精度算出

In [ ]:
model_full_acc = accuracy(test_x_full, test_y_full, model_full)
print("model_base accuracy :", model_base_acc)
print("model_drop accuracy :", model_drop_acc)
print("model_full accuracy :", model_full_acc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


model_base accuracy : 0.6795392953929539
model_drop accuracy : 0.7730352303523035
model_full accuracy : 0.9092140921409214


タイトル+本文を入力データとすることで精度が大きく向上した。

本文はタイトルよりも文字数が非常に多く、特徴的な単語が多く含まれていることが、精度の向上に寄与していると考えられる。

また、LSTMによって系列の長い本文データであっても学習できることがわかる。

# 【モデルの改良】

続いて，タイトル+本文を入力データとし、モデルをさらに改良した場合に精度が向上するか調査する。
モデルには以下の2つの改良を加える。

①LSTM層の多層化（LSTM層を3層積層する）

②LSTM層のBidirectional化


改良モデル定義

In [ ]:
# 改良モデル
class DeepLSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size):
        super(DeepLSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout1 = nn.Dropout(0.5)
        # ①LSTM層を3層積層、②Bidirectional化
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=3, batch_first=True, dropout=0.5, bidirectional=True)
        self.dropout2 = nn.Dropout(0.5)
        # Bidirectional化で隠れユニット次元数が倍増する
        self.hidden2tag = nn.Linear(hidden_dim * 2, target_size)
        self.softmax = nn.LogSoftmax()
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        embeds = self.dropout1(embeds)
        _, bilstm_hc = self.lstm(embeds)
        bilstm_out = torch.cat([bilstm_hc[0][-2], bilstm_hc[0][-1]], dim=1)
        drop_out = self.dropout2(bilstm_out)
        tag_space = self.hidden2tag(drop_out)
        tag_scores = self.softmax(tag_space.squeeze())

        return tag_scores

EMBEDDING_DIM = 200
HIDDEN_DIM = 128
VOCAB_SIZE = len(word2index)
TAG_SIZE = len(categories)

model_deep = DeepLSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE).to(device)
loss_function_deep = nn.NLLLoss()
optimizer_deep = optim.Adam(model_deep.parameters(), lr=0.001)

改良モデルの学習（入力データはタイトル+本文）

In [ ]:
trainer(train_x_full, train_y_full, model_deep, loss_function_deep, optimizer_deep)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 0 	 loss 301.7329161763191
epoch 1 	 loss 193.9149765074253
epoch 2 	 loss 147.99074018001556
epoch 3 	 loss 137.1787267625332
epoch 4 	 loss 106.96926374733448
epoch 5 	 loss 102.14768919348717
epoch 6 	 loss 96.81617532670498
epoch 7 	 loss 81.20886664092541
epoch 8 	 loss 69.31662288308144
epoch 9 	 loss 67.86532012373209
epoch 10 	 loss 67.98675994575024
epoch 11 	 loss 65.67561437934637
epoch 12 	 loss 51.92248048633337
epoch 13 	 loss 46.50503298267722
epoch 14 	 loss 43.253642812371254
epoch 15 	 loss 65.30806917324662
epoch 16 	 loss 54.64137262851
epoch 17 	 loss 39.730345241725445
epoch 18 	 loss 36.79890500754118
epoch 19 	 loss 32.542437890544534
epoch 20 	 loss 40.21688421629369
epoch 21 	 loss 34.443190230987966
epoch 22 	 loss 28.707818317227066
epoch 23 	 loss 34.03942531673238
epoch 24 	 loss 30.04968861490488
epoch 25 	 loss 25.109764070250094
epoch 26 	 loss 22.695261615794152
epoch 27 	 loss 25.481391134671867
epoch 28 	 loss 24.98621327150613
epoch 29 	 loss 

改良モデル推論・精度算出

In [ ]:
model_deep_acc = accuracy(test_x_full, test_y_full, model_deep)
print("model_base accuracy :", model_base_acc)
print("model_drop accuracy :", model_drop_acc)
print("model_full accuracy :", model_full_acc)
print("model_deep accuracy :", model_deep_acc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


model_base accuracy : 0.6795392953929539
model_drop accuracy : 0.7730352303523035
model_full accuracy : 0.9092140921409214
model_deep accuracy : 0.9153116531165312


In [ ]:
model_full_acc_train = accuracy(train_x_full, train_y_full, model_full)
print("model_full train accuracy :", model_full_acc_train)
model_deep_acc_train = accuracy(train_x_full, train_y_full, model_deep)
print("model_deep train accuracy :", model_deep_acc_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


model_full train accuracy : 0.9820338983050847


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


model_deep train accuracy : 0.995593220338983


モデルの改良によりわずかに精度が向上したが、学習時間（約4時間）の割に大きな改善は見られなかった。

改良モデルの学習データに対する精度を見ると、99%以上であり、過学習気味であることがわかる。これはモデルの複雑さが増したことに由来するものと考えられる。

# 【まとめ】

今回はLSTMを用いた文書分類にチャレンジした。

その結果、ベースモデルでは過学習を引き起こしていることがわかった。そこでドロップアウトをモデルに適用したところ、過学習を抑制し、汎化性能を向上させることができた。

また、今回はニュース記事の分類を行ったが、記事のタイトルのみならず、本文も入力データとすることで約90%の精度を出すことができた。
これは、本文中に特徴的な単語が多数含まれていること、LSTMが長い系列データに対しても学習可能であることが要因と考えられる。

一方でLSTMモデルの多層化、Bidirectional化を試したが、大きな改善は見られなかった。細かいチューニングによって改善する可能性はあると思われる。




---

参考文献

[1]PyTorchを使ってLSTMで文章分類を実装してみた（バッチ化対応ver）https://qiita.com/m__k/items/db1a81bb06607d5b0ec5

[2]PyTorchのBidirectional LSTMのoutputの仕様を確認してみた
https://qiita.com/m__k/items/78a5125d719951ca98d3